# CHA Scrapper

1. This scrapper mines data from www.icegate.gov.in for missing CHA Names of Bill of Entry(Import).

2. Select the Excel file where :
    1. Location (Port Name) in Column B
    2. Bill of Entry in Column C
    3. Bill of Entry Date in Column D
    4. Log as NULL (Scrapped CHA Name will be store in this Column D)
   
 
3. Enter the Captcha text and click on Chrome Driver to pass the active instance as Chrome.

In [ ]:
import xlwings as xw
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as cond
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
from tqdm import tqdm_notebook
from selenium.common.exceptions import*
# from selenium.common.exceptions import NoSuchElementException
from tkinter.filedialog import askopenfilename
from tkinter import*
import win32com.client as win


# window to select the excel file containing data
root =Tk()
root.update()
path_file = filedialog.askopenfilename(initialdir = 'C://Users//data2//Desktop//IEC To Fetch', title = "Select your Excel file that contains input data:") 
root.destroy()


# load Workbook and Sheet
workbook= xw.Book(path_file)
# sheet = workbook["Sheet1"]  
sheet = workbook.sheets[0]

# function to read Data from Excel Sheet
def readData(column_name, rownum):
    rownum1 = str(rownum) 
    cell = column_name+rownum1
    return sheet.range(cell).value
#     return sheet.cell(row = rownum, column = columnno).value

# function to write Data from Excel Sheet
def writeData(column_name, rownum, data):
    rownum1 = str(rownum)
    cell = column_name+ str(rownum)
    sheet.range(cell).value= data

# #path to the chromedriver
# options = webdriver.FirefoxOptions()
# options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(executable_path="chromedriver.exe") 
# driver = webdriver.Firefox(executable_path="C://Users//Manish//Downloads//webdriver_win32//geckodriver.exe") 
driver.set_page_load_timeout(10)


#url to the website data to be retrieve
driver.get("https://enquiry.icegate.gov.in/enquiryatices/beTrackIces")

In [ ]:
captchatxt = input("Enter the captcha: ")

#send data to the textbox
rows = sheet.range('A' + str(sheet.cells.last_cell.row)).end('up').row
count = 0
cap = 0
for x in tqdm_notebook(range(42000,rows+1), desc = "Progress..."):
    log = readData("E", x)
    driver.set_page_load_timeout(5)
    if log == "NULL":
        if cap>139:
            driver.get("https://enquiry.icegate.gov.in/enquiryatices/beTrackIces")
            captchatxt = input("Enter the captcha: ")
            cap =0
            
        location = readData("B", x)
        be_date = readData("D",x)
        be_num = readData("C",x)
        be_num = int(be_num)
        
        
        try:
            WebDriverWait(driver, 10).until(cond.presence_of_element_located((By.ID, "location"))).send_keys(location)
        except:
            driver.forward()
            time.sleep(2)
            beDATE = driver.find_element_by_id('beDATE')
            driver.execute_script("arguments[0].removeAttribute('readonly')", beDATE)
            beDATE.send_keys(be_date)
            submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
            submit.click()
            
            try:
                submit1 = WebDriverWait(driver, 10).until(cond.element_to_be_clickable((By.CLASS_NAME, "page_button")))
                submit1.click()
            except:
                driver.set_page_load_timeout(3)
                while home is None and retry<15:
                    try:
                        submit1 = WebDriverWait(driver, 1).until(cond.element_to_be_clickable((By.CLASS_NAME, "page_button")))
                        submit1.click()
                        home= WebDriverWait(driver, 1).until(cond.presence_of_element_located((By.CSS_SELECTOR, "#nav > li.current > a"))).text.split("&")
                    except:
                        wsh= win.Dispatch("WScript.Shell")
                        wsh.AppActivate("Chrome")
                        wsh.SendKeys('{F5}')
                        time.sleep(1)
                        retry+=1
                        cap = cap+1
                        
                
                try:
                    cha =WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[4]'))).text.split("&")[0]
                    typex =WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[3]'))).text.split("&")[0]
                    iec = WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[1]'))).text.split("&")[0]
                    writeData("E", x, cha) 
                    writeData("F", x, typex)
                    writeData("G", x, iec)
                    driver.set_page_load_timeout(10)
                    driver.implicitly_wait(1) 
                    driver.back()
                    continue
                except:
                    writeData("E", x, 'Sorry :(')
                    count = count+1
                    driver.implicitly_wait(1)
                    driver.back()
                    cap = cap+1
                    continue
                   

                    
        try:
            driver.find_element_by_id('beNO').send_keys(be_num)
        except:
            try:
                cha =WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[4]'))).text.split("&")[0]
                typex =WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[3]'))).text.split("&")[0]
                iec = WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[1]'))).text.split("&")[0]
                writeData("E", x, cha) 
                writeData("F", x, typex)
                writeData("G", x, iec)
                driver.set_page_load_timeout(10)
                driver.implicitly_wait(1) 
                driver.back()
                continue
            except:
                writeData("E", x, 'Sorry :(')
                count = count+1
                driver.implicitly_wait(1)
                driver.back()
                cap = cap+1
                driver.back()
                continue
                
            
   

    #send date after remove calender
        beDATE = driver.find_element_by_id('beDATE')
        driver.execute_script("arguments[0].removeAttribute('readonly')", beDATE)
        beDATE.send_keys(be_date)

        
    # send the captcha entered manually
        driver.find_element_by_id('captchaResp').send_keys(captchatxt)
        
        
#     # click submit button

        submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
        try:
            submit.click()
        except:
            driver.back()
            time.sleep(2)
            continue
                        
                        
        try:
            submit1 = WebDriverWait(driver, 10).until(cond.element_to_be_clickable((By.CLASS_NAME, "page_button")))
            submit1.click()
        except:
            try:
#                 submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
#                 submit.click()
#                 driver.set_page_load_timeout(3)
#                 driver.forward()
#                 driver.back()
                submit = WebDriverWait(driver,5).until(cond.element_to_be_clickable((By.ID,'SubB')))
                submit.click()
            except:
                home= None
                retry = 0
                driver.set_page_load_timeout(3)
                while home is None and retry<15:
                    try:
                        submit1 = WebDriverWait(driver, 1).until(cond.element_to_be_clickable((By.CLASS_NAME, "page_button")))
                        submit1.click()
                        home= WebDriverWait(driver, 1).until(cond.presence_of_element_located((By.CSS_SELECTOR, "#nav > li.current > a"))).text.split("&")
                    except:
                        wsh= win.Dispatch("WScript.Shell")
                        wsh.AppActivate("Chrome")
                        wsh.SendKeys('{F5}')
                        time.sleep(1)
                        retry+=1
                        cap = cap+1
             

        
        try:
            cha =WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[4]'))).text.split("&")[0]
            typex =WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[3]'))).text.split("&")[0]
            iec = WebDriverWait(driver, 2).until(cond.presence_of_element_located((By.XPATH, '//*[@id="beICES_Details"]/center/div/table/tbody/tr[2]/td[1]'))).text.split("&")[0]

        except:
            writeData("E", x, 'Sorry :(')
            count = count+1
            driver.implicitly_wait(1)
            driver.back()
            cap = cap+1
            continue
            
#         try:
#             temp_fob = driver.find_element_by_css_selector("#sbICES_Details > center > div > table > tbody > tr.tdText > td:nth-child(8)").text.split("&")
#            # temp_total =WebDriverWait(driver, 1).until(cond.presence_of_element_located((By.CSS_SELECTOR, "#beICES_Details > center > div > table > tbody > tr:nth-child(2) > td:nth-child(2)"))).text.split("&")
#         except:
#             pass
#         fob = temp_fob[0]
        writeData("E", x, cha) 
        writeData("F", x, typex)
        writeData("G", x, iec)
        driver.set_page_load_timeout(10)
        driver.implicitly_wait(1)
        cap = 0
        driver.back()
    else:
        continue
# In[ ]:
driver.quit()
writeData("K", 11, "No record found for {} items".format(count))
print("No record found for {} items".format(count))
workbook.save(path_file)
driver.close()